In [1]:
import numpy as np
import math
import random

In [2]:
def stringToNums(string, invDict):
    parsed = [string.upper()[i] for i in range(len(string.upper()))]
    nums = [invDict[i] for i in parsed]
    return nums

def encode(x,binDig):
    encoded = []
    for i in range(binDig,0,-1):
        if x < 2**(i-1):
            encoded.append(0)
            continue
        else:
            encoded.append(x // (2**(i-1)))
            x = x % (2**(i-1))
    encoded = np.array(encoded)
    return encoded

def encodeLoop(x, binDig):
    for i in range(len(x)):
        if i == 0:
            codedArray = np.array(encode(x[i],binDig))
        else:
            codedArray = np.vstack([codedArray,encode(x[i],binDig)])
    return codedArray

def decode(x):
    pwrs = np.array([2**(j-1) for j in range(len(x[0]),0,-1)])
    for i in range(len(x)):
        if i == 0:
            decoded = np.array([pwrs @ x[i]])
        else:
            decoded = np.hstack([decoded, np.array(pwrs @ x[i])])
    decoded = list(decoded)
    return decoded

def decodeToString(x, dict):
    chars = [dict[i] for i in x]
    frankenstring = ''.join(chars)
    return frankenstring

def randOrder(x):
    a = list(range(1,x+1))
    rs = []
    for i in range(len(a)):
        r = random.choice(a)
        a.remove(r)
        rs.append(r)
    return rs

def scrambleT(length):
    ro = randOrder(length)
    for i in range(len(ro)):
        if i == 0:
            T = np.array([1 if a == ro[i]-1 else 0 for a in range(len(ro))])
        else:
            T = np.vstack([T, np.array([1 if a == ro[i]-1 else 0 for a in range(len(ro))])])
    return T

def scrambleChain(string, dict, invDict, keyMatrix, binDigits=6):
    K = keyMatrix
    scrambled = decodeToString(decode((K @ encodeLoop(stringToNums(string, invDict), binDigits).T).T),dict)
    return scrambled

def unscrambleChain(string, dict, invDict, keyMatrix, binDigits=6):
    K = keyMatrix
    Kinv = np.linalg.inv(keyMatrix)
    unscrambled = decodeToString(decode((Kinv @ encodeLoop(stringToNums(string, invDict),binDigits).T).T),dict)
    return unscrambled

In [3]:
ascii_dict = {
    0: '_',
    1: ' ',
    2: '!',
    3: '"',
    4: '#',
    5: '$',
    6: '%',
    7: '&',
    8: '’',
    9: '(',
    10: ')',
    11: '*',
    12: '+',
    13: ',',
    14: '-',
    15: '.',
    16: '/',
    17: '0',
    18: '1',
    19: '2',
    20: '3',
    21: '4',
    22: '5',
    23: '6',
    24: '7',
    25: '8',
    26: '9',
    27: ':',
    28: ';',
    29: '<',
    30: '=',
    31: '>',
    32: '?',
    33: '@',
    34: 'A',
    35: 'B',
    36: 'C',
    37: 'D',
    38: 'E',
    39: 'F',
    40: 'G',
    41: 'H',
    42: 'I',
    43: 'J',
    44: 'K',
    45: 'L',
    46: 'M',
    47: 'N',
    48: 'O',
    49: 'P',
    50: 'Q',
    51: 'R',
    52: 'S',
    53: 'T',
    54: 'U',
    55: 'V',
    56: 'W',
    57: 'X',
    58: 'Y',
    59: 'Z',
    60: '[',
    61: '\\',
    62: ']',
    63: '^',
    }

ascii_inv_dict = {j:i for (i,j) in ascii_dict.items()}

In [4]:
string = 'My name is Sam'

In [5]:
T = scrambleT(6)
#T = np.eye(6)
#T = np.array([[0, 0, 0, 0, 0, 1],
#              [0, 0, 0, 0, 1, 0],
#              [0, 0, 0, 1, 0, 0],
#              [0, 0, 1, 0, 0, 0],
#              [0, 1, 0, 0, 0, 0],
#              [1, 0, 0, 0, 0, 0]])
T

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [6]:
Tcoded = decodeToString(decode(T), ascii_dict)
Tcoded

'#/ !’?'

In [7]:
Tdecoded = encodeLoop(stringToNums(Tcoded, ascii_inv_dict), 6)
Tdecoded

array([[0, 0, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0],
       [0, 0, 1, 0, 0, 0],
       [1, 0, 0, 0, 0, 0]])

In [8]:
encrypted = scrambleChain(string, ascii_dict, ascii_inv_dict, T)
encrypted

'F6’N$FD’&P’P$F'

In [9]:
decrypted = unscrambleChain(encrypted, ascii_dict, ascii_inv_dict, T)
decrypted

'MY NAME IS SAM'